In [48]:
import mysql.connector as db
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

class db_communication:
    """
    ____________________________________________________________________________________________
    Benötigte Python-Module:
    - mysql
    - mysql-connector
    - pandas
    - numpy
    ____________________________________________________________________________________________
    Bietet eine API zur Kommnunikation mit der Fahrrad-Shop-Datenbank.
    Auf Basis des mysql-Moduls wird mit einer MariaDB kommuniziert.
    
    Durch print()-Ausgaben wird der Erfolg oder Misserfolg bei einem Funktionsaufruf ausgegeben.
    
    Die Klasse ist wie folgt zu nutzen:
    my_db = db_communication()
    -----
    my_db.user_func_xyz(args)
    -----
    my_db.close()
    """
    def __init__(self, config = None):
        """
        Default (config = None) wird die Verbindung zur Standard-Datenbank hergestellt.
        Alternativ kann ein config-dict übergeben werden
        config = {
            'user': '',
            'password': '',
            'host': '',
            'port': '',
            'database': ''
        }
        """
        self.__db = self.connect(config)
        if self.__db:
            self.__cursor = self.__db.cursor()
            print("Successfully connected.")
        else:
            print("... no connection ...")

    def connect(self, config=None) -> db.MySQLConnection:    
        """
        Stellt die Verbindung zur Datenbank her und nutzt die übergeben Parameter config.
        Return: 'db.MySQLConnection'.
        """
        if not config:
            config = {
            'user': 'root',
            'password': 'FDS-apm1',
            'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
            'port': '3306',
            'database': 'NeueDB'
            }

        try:
            return mysql.connector.connect(**config)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            print("Not connected")
            
    def close(self):
        """ 
        Schließt die Verbindung zur Datenbank.
        """
        if self.__db:
            self.__db.close()
            print("Connection closed")
        else:
            pass
    
    def read_tables(self) -> list:
        """
        Liest die Tabellen der Datenbank aus und gibt diese in Listen-Form zurück.
        """
        if self.__db:
            self.__cursor.execute("SHOW TABLES")

            tables = list()
            for x in self.__cursor:
                tables.append(x)

            return tables
        else:
            pass
        
    def insert_data(self, table_name, data_as_dict):
        """
        Der Tabelle 'table_name' wird eine neue Zeile hinzugefügt. 
        Der Inhalt der Zeile wird aus dem 'data_as_dict' erstellt. 
        Die Spalentennamen sind die 'keys' und die Zellenwerte die 'values' des Dictionaries.
        """
        try:
            sql = "INSERT INTO `" + table_name + "` ("
            values = ""
            for key in data_as_dict:
                sql += "`" + str(key) + "`, "
                if data_as_dict[key]: 
                    values += "'" + str(data_as_dict[key]) + "', "
                else:
                    values += "NULL , "
            sql = sql[:len(sql)-2]
            values = values[:len(values)-2]
            sql += ") VALUES (" + values + ");"
            
            print(sql)
            self.__cursor.execute(sql)
            self.__db.commit()
            return True
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def get_table(self, table_name, key = None, column = None) -> pd.DataFrame:
        """
        Die gewünschte Tabelle 'table_name' wird als pd.DataFrame zurückgegeben.
        """
        if key and column:
            pass # was war hier nochmal?
        elif column:
            self.__cursor.execute("SELECT {} FROM `{}`".format(column, table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            return pd.DataFrame(np.array(rows), columns = [column])
        elif key:
            pass # was war hier nochmal?
        else:
            self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(table_name))
            columns = []
            for y in self.__cursor:
                columns.append(y[0])
            self.__cursor.execute("SELECT * FROM `{}`".format(table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            try: 
                df = pd.DataFrame(np.array(rows), columns=columns)
                return df
            except ValueError:
                return None
        
    def drop_table(self, name) -> bool:
        """
        Die Tabelle name wird gelöscht, allerdings nur unter der Bedingung, 
        dass die Tabelle keine Abhängigkeiten zu anderen Tabellen hat.
        """
        sql = "DROP TABLE %s" %str(name)
        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Drop table: ", str(name))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
        
        return True

    def show_columns(self, table_name):
        """
        Gibt die Spalten der gewünschten Tabelle über die print-Ausgabe aus.
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            return self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(x))
        else:
            print("Table", name, "already exists.")
            return False
        
    def storno(self, key):
        """
        Datenbank spezifische Funktion.
        Eine existierende Bestellung wird storniert, identifiziert an dem 'key' == 'Bestellung.id'. 
        Die Spalte "Storniert" wird auf True gesetzt.
        """
        
        sql = "UPDATE `Bestellungen` SET `Storniert`='1' WHERE `Bestellungen.id`= '" + key + "';"

        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Update Bestellung:", str(key))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def create_table(self, name, rows) -> bool:
        """
        Erstellt eine Tabelle mit dem Namen 'name' und den Spalten 'rows'.
        'rows'-Elemente müssen im MariaDB-Format die Einstellungen enthalten. 
        Bsp.1: 
        `Bauteil.id` INT NOT NULL AUTO_INCREMENT
        Bsp.2:
        FOREIGN KEY (`Baugruppe`) REFERENCES `Baugruppen`(`Baugruppe`)
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            sql = "CREATE TABLE `" + name + "` ("
            for row in rows:     
                if rows.index(row) == (len(rows)-1):
                    sql += " " + row
                else:
                    sql += " " + row + ","
            sql += ")"
            print(sql)
            self.__cursor.execute(sql)
            print("Created table: ", name)
            return True
        else:
            print("Table", name, "already exists.")
            return False

In [ ]:
import os
from path import Path
path = os.getcwd()
path = Path(path) / "Bauteile_ready.csv"
print(path)
df = pd.read_csv(path, sep=',')
df.dropna(axis = 1, inplace=True, how = "all")
#print(df)

columns = [df.columns][0]
column = ()
for i in range(len(columns)):
     column += (columns[i],)

my_db = db_communication()

data = df.values.tolist()

for i in range(0,len(data)):
    data_as_dict = {}
    for j in range(len(column)):
            data_as_dict[column[j]] = data[i][j]
    my_db.insert_data("Bauteile", data_as_dict)

my_db.close()

In [71]:
config_kunde = ["`KundenNr` INTEGER NOT NULL AUTO_INCREMENT",
                "`Name` CHAR(30) NOT NULL",
                "`Vorname` CHAR(30) NOT NULL",
                "`Mail` CHAR(60)",
                "PRIMARY KEY (`KundenNr`)"]
my_db = db_communication()
my_db.create_table("Kunde", config_kunde)
my_db.close()

Successfully connected.
CREATE TABLE `Kunde` ( `KundenNr` INTEGER NOT NULL AUTO_INCREMENT, `Name` CHAR(30) NOT NULL, `Vorname` CHAR(30) NOT NULL, `Mail` CHAR(60), PRIMARY KEY (`KundenNr`))
Created table:  Kunde
Connection closed


In [72]:
config_auftrag = [ "`AuftrNr` INTEGER NOT NULL AUTO_INCREMENT",
                   "`Bestelldatum` DATE",
                   "`Lieferdatum` DATE",
                   "`KundenNr` INTEGER NOT NULL",
                   "`Gesamtpreis` NUMERIC(7,2)",
                   "PRIMARY KEY (`AuftrNr`)",
                   "FOREIGN KEY (`KundenNr`) REFERENCES `Kunde`(`KundenNr`)"]
my_db = db_communication()
my_db.create_table("Auftrag", config_auftrag)
my_db.close()

Successfully connected.
CREATE TABLE `Auftrag` ( `AuftrNr` INTEGER NOT NULL AUTO_INCREMENT, `Bestelldatum` DATE, `Lieferdatum` DATE, `KundenNr` INTEGER NOT NULL, `Gesamtpreis` NUMERIC(7,2), PRIMARY KEY (`AuftrNr`), FOREIGN KEY (`KundenNr`) REFERENCES `Kunde`(`KundenNr`))
Created table:  Auftrag
Connection closed


In [73]:
config_merkmalcluster = [ "`MerkmalNr` INTEGER NOT NULL AUTO_INCREMENT",
                          "`Merkmalsbezeichnung`CHAR(35)",
                          "PRIMARY KEY (`MerkmalNr`)"]

my_db = db_communication()
my_db.create_table("Merkmalcluster", config_merkmalcluster)
my_db.close()

Successfully connected.
CREATE TABLE `Merkmalcluster` ( `MerkmalNr` INTEGER NOT NULL AUTO_INCREMENT, `Merkmalsbezeichnung`CHAR(35), PRIMARY KEY (`MerkmalNr`))
Created table:  Merkmalcluster
Connection closed


In [74]:
config_merkmale = [ "`ArtNr` INTEGER NOT NULL AUTO_INCREMENT",
                    "`Merkmalcluster` INTEGER NOT NULL",
                    " Bezeichnung CHAR(35) NOT NULL",
                    "`Preis` NUMERIC(7,2)",
                    "`Montagezeit` INTEGER",
                    "`Ausstattung` CHAR(35)",
                    "PRIMARY KEY (`ArtNr`)"]

my_db = db_communication()
my_db.create_table("Merkmale", config_merkmale)
my_db.close()

Successfully connected.
CREATE TABLE `Merkmale` ( `ArtNr` INTEGER NOT NULL AUTO_INCREMENT, `Merkmalcluster` INTEGER NOT NULL,  Bezeichnung CHAR(35) NOT NULL, `Preis` NUMERIC(7,2), `Montagezeit` INTEGER, `Ausstattung` CHAR(35), PRIMARY KEY (`ArtNr`))
Created table:  Merkmale
Connection closed


In [75]:
config_einzelteile = [ "`EinzelteilNr` INTEGER NOT NULL AUTO_INCREMENT",
                       " Bezeichnung CHAR(35) NOT NULL",
                       "`Preis` NUMERIC(7,2)",
                       "PRIMARY KEY (`EinzelteilNr`)"]

my_db = db_communication()
my_db.create_table("Einzelteile", config_einzelteile)
my_db.close()

Successfully connected.
CREATE TABLE `Einzelteile` ( `EinzelteilNr` INTEGER NOT NULL AUTO_INCREMENT,  Bezeichnung CHAR(35) NOT NULL, `Preis` NUMERIC(7,2), PRIMARY KEY (`EinzelteilNr`))
Created table:  Einzelteile
Connection closed


In [76]:
config_konfigurator = [ "`KonfigNr` INTEGER NOT NULL AUTO_INCREMENT",
                        "`AuftragNr` INTEGER NOT NULL",
                        "`ArtikelNr` INTEGER NOT NULL",
                        "PRIMARY KEY (`KonfigNr`)",
                        "FOREIGN KEY (`AuftragNr`) REFERENCES `Auftrag`(`AuftrNr`)",
                        "FOREIGN KEY (`ArtikelNr`) REFERENCES `Merkmale`(`ArtNr`)"]
my_db = db_communication()
my_db.create_table("Konfigurator", config_konfigurator)
my_db.close()

Successfully connected.
CREATE TABLE `Konfigurator` ( `KonfigNr` INTEGER NOT NULL AUTO_INCREMENT, `AuftragNr` INTEGER NOT NULL, `ArtikelNr` INTEGER NOT NULL, PRIMARY KEY (`KonfigNr`), FOREIGN KEY (`AuftragNr`) REFERENCES `Auftrag`(`AuftrNr`), FOREIGN KEY (`ArtikelNr`) REFERENCES `Merkmale`(`ArtNr`))
Created table:  Konfigurator
Connection closed


In [77]:
config_stueckliste = [ "`ArtNr` INTEGER",
                       " `EinzelteilNr` INTEGER",
                       "`Anzahl` INTEGER",
                       "FOREIGN KEY (`ArtNr`) REFERENCES `Merkmale`(`ArtNr`)",
                       "FOREIGN KEY (`EinzelteilNr`) REFERENCES `Einzelteile`(`EinzelteilNr`)",
                       "PRIMARY KEY (`ArtNr`,`EinzelteilNr`)"]
my_db = db_communication()
my_db.create_table("Stueckliste", config_stueckliste)
my_db.close()

Successfully connected.
CREATE TABLE `Stueckliste` ( `ArtNr` INTEGER,  `EinzelteilNr` INTEGER, `Anzahl` INTEGER, FOREIGN KEY (`ArtNr`) REFERENCES `Merkmale`(`ArtNr`), FOREIGN KEY (`EinzelteilNr`) REFERENCES `Einzelteile`(`EinzelteilNr`), PRIMARY KEY (`ArtNr`,`EinzelteilNr`))
Created table:  Stueckliste
Connection closed


In [78]:
#Fuelle DB 
kunde = [{'KundenNr' : '1','Name' : 'Kraus','Vorname':'Maria','Mail' : 'MaKrau@radelwelt.com'}]
my_db = db_communication()
for i in range(len(kunde)):
    my_db.insert_data("Kunde", kunde[i])
my_db.close()

Successfully connected.
INSERT INTO `Kunde` (`KundenNr`, `Name`, `Vorname`, `Mail`) VALUES ('1', 'Kraus', 'Maria', 'MaKrau@radelwelt.com');
Connection closed


In [79]:
auftrag = [{'AuftrNr':'1','Bestelldatum':'2020-04-20','Lieferdatum':'2020-06-02','Gesamtpreis': '1313.54','KundenNr':'1'}]

my_db = db_communication()
for i in range(len(auftrag)):
    my_db.insert_data("Auftrag", auftrag[i])
my_db.close()

Successfully connected.
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `Gesamtpreis`, `KundenNr`) VALUES ('1', '2020-04-20', '2020-06-02', '1313.54', '1');
Connection closed


In [80]:
merkmalcluster = [ {'MerkmalNr':'1','Merkmalsbezeichnung':'Beleuchtung'},
                   {'MerkmalNr':'2','Merkmalsbezeichnung':'Bremsen'},
                   {'MerkmalNr':'3','Merkmalsbezeichnung':'Felgen'},
                   {'MerkmalNr':'4','Merkmalsbezeichnung':'Gangschaltung'},
                   {'MerkmalNr':'5','Merkmalsbezeichnung':'Lenker'},
                   {'MerkmalNr':'6','Merkmalsbezeichnung':'Rahmen'},
                   {'MerkmalNr':'7','Merkmalsbezeichnung':'Reifen'},
                   {'MerkmalNr':'8','Merkmalsbezeichnung':'Sattel'}]

my_db = db_communication()
for i in range(len(merkmalcluster)):
    my_db.insert_data("Merkmalcluster", merkmalcluster[i])
my_db.close()

Successfully connected.
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('1', 'Beleuchtung');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('2', 'Bremsen');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('3', 'Felgen');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('4', 'Gangschaltung');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('5', 'Lenker');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('6', 'Rahmen');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('7', 'Reifen');
INSERT INTO `Merkmalcluster` (`MerkmalNr`, `Merkmalsbezeichnung`) VALUES ('8', 'Sattel');
Connection closed


In [81]:
merkmale = [ { 'ArtNr':'100001','Merkmalcluster':'1','Bezeichnung': 'Beleuchtung Dynamo','Preis':'5.00','Montagezeit': '1','Ausstattung':'City'},
             { 'ArtNr':'100002','Merkmalcluster':'1','Bezeichnung': 'Beleuchtung LED','Preis':'29.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'100003','Merkmalcluster':'1','Bezeichnung': 'Leuchtstreifen','Preis':'15.00','Montagezeit': '0','Ausstattung':'City'},
             { 'ArtNr':'100004','Merkmalcluster':'1','Bezeichnung': 'Keine Beleuchtung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'200001','Merkmalcluster':'2','Bezeichnung': 'Shimano V-Brake','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'200002','Merkmalcluster':'2','Bezeichnung': 'Clarks Scheibenbremse','Preis':'69.99','Montagezeit': '2','Ausstattung':'Sport'},
             { 'ArtNr':'200003','Merkmalcluster':'2','Bezeichnung': 'Shimano GRB Scheibenbremse','Preis':'99.99','Montagezeit': '2','Ausstattung':'NULL'},
             { 'ArtNr':'200004','Merkmalcluster':'2','Bezeichnung': 'Fagura Felgenbremse','Preis':'39.99','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'200005','Merkmalcluster':'2','Bezeichnung': 'Keine Bremse','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'300001','Merkmalcluster':'3','Bezeichnung': 'Aluminium Felgen','Preis':'10.00','Montagezeit': '3','Ausstattung':'NULL'},
             { 'ArtNr':'300002','Merkmalcluster':'3','Bezeichnung': 'Redondo Rennradfelgen','Preis':'99.99','Montagezeit': '4','Ausstattung':'Rennen'},
             { 'ArtNr':'300003','Merkmalcluster':'3','Bezeichnung': 'Spengle Carbonfelgen','Preis':'79.99','Montagezeit': '2','Ausstattung':'NULL'},
             { 'ArtNr':'300004','Merkmalcluster':'3','Bezeichnung': 'Taylor Sportfelgen','Preis':'39.99','Montagezeit': '5','Ausstattung':'Sport'},
             { 'ArtNr':'300005','Merkmalcluster':'3','Bezeichnung': 'Taylor Cityfelgen','Preis':'69.99','Montagezeit': '5','Ausstattung':'City'},
             { 'ArtNr':'300006','Merkmalcluster':'3','Bezeichnung': 'Keine Felge','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'400001','Merkmalcluster':'4','Bezeichnung': '3 Gang Nexus RT','Preis':'100.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'400002','Merkmalcluster':'4','Bezeichnung': '7 Gang Nexus RT','Preis':'110.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'400003','Merkmalcluster':'4','Bezeichnung': '9 Gang Nexus RT','Preis':'150.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'400004','Merkmalcluster':'4','Bezeichnung': 'Keine Schaltung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'500001','Merkmalcluster':'5','Bezeichnung': 'Retro Lenker','Preis':'89.99','Montagezeit': '1','Ausstattung':'Sport'},
             { 'ArtNr':'500002','Merkmalcluster':'5','Bezeichnung': 'Classic Lenker','Preis':'59.49','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'500003','Merkmalcluster':'5','Bezeichnung': 'Rennradlenker','Preis':'99.99','Montagezeit': '1','Ausstattung':'Rennen'},
             { 'ArtNr':'500004','Merkmalcluster':'5','Bezeichnung': 'Kein Lenker','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'600001','Merkmalcluster':'6','Bezeichnung': 'FH100','Preis':'499.00','Montagezeit': '2','Ausstattung':'City'},
             { 'ArtNr':'600002','Merkmalcluster':'6','Bezeichnung': 'S100','Preis':'649.99','Montagezeit': '2','Ausstattung':'Sport'},
             { 'ArtNr':'600003','Merkmalcluster':'6','Bezeichnung': 'Trofeo 5','Preis':'599.00','Montagezeit': '2','Ausstattung':'Rennen'},
             { 'ArtNr':'600004','Merkmalcluster':'6','Bezeichnung': 'Kein Rahmen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'700001','Merkmalcluster':'7','Bezeichnung': 'Bereifung Schwalbe City','Preis':'15.00','Montagezeit': '2','Ausstattung':'City'},
             { 'ArtNr':'700002','Merkmalcluster':'7','Bezeichnung': 'Bereifung Schwalbe Sport','Preis':'20.00','Montagezeit': '1','Ausstattung':'Sport'},
             { 'ArtNr':'700003','Merkmalcluster':'7','Bezeichnung': 'Bereifung Schwalbe Marathon','Preis':'35.00','Montagezeit': '4','Ausstattung':'Rennen'},
             { 'ArtNr':'700004','Merkmalcluster':'7','Bezeichnung': 'Keine Reifen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
             { 'ArtNr':'800001','Merkmalcluster':'8','Bezeichnung': 'Asvert Cityradsattel','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'800002','Merkmalcluster':'8','Bezeichnung': 'Asvert Comfortsattel','Preis':'39.00','Montagezeit': '1','Ausstattung':'NULL'},
             { 'ArtNr':'800003','Merkmalcluster':'8','Bezeichnung': 'Avert Rennradsattel','Preis':'59.00','Montagezeit': '1','Ausstattung':'Rennen'},
             { 'ArtNr':'800004','Merkmalcluster':'8','Bezeichnung': 'Kein Sattel','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'}]
my_db = db_communication()
for i in range(len(merkmale)):
    my_db.insert_data("Merkmale", merkmale[i])
my_db.close()

Successfully connected.
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100001', '1', 'Beleuchtung Dynamo', '5.00', '1', 'City');
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100002', '1', 'Beleuchtung LED', '29.00', '1', 'NULL');
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100003', '1', 'Leuchtstreifen', '15.00', '0', 'City');
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('100004', '1', 'Keine Beleuchtung', '0.00', '0', 'NULL');
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('200001', '2', 'Shimano V-Brake', '10.00', '1', 'NULL');
INSERT INTO `Merkmale` (`ArtNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`, `Montagezeit`, `Ausstattung`) VALUES ('200002', '

In [82]:
einzelteile = [{ 'EinzelteilNr':'1001','Bezeichnung': 'Rückstrahler','Preis':'5.00'},
{ 'EinzelteilNr':'2001','Bezeichnung': 'Bremsgriff','Preis':'9.99'},
{ 'EinzelteilNr':'4001','Bezeichnung': 'Zahnkranz','Preis':'50.00'},
{ 'EinzelteilNr':'4002','Bezeichnung': 'Kette','Preis':'9.00'},
{ 'EinzelteilNr':'4003','Bezeichnung': 'Kettenblätter','Preis':'3.99'},
{ 'EinzelteilNr':'4004','Bezeichnung': 'Pedale','Preis':'8.99'},
{ 'EinzelteilNr':'4005','Bezeichnung': 'Innenlager','Preis':'3.99'},
{ 'EinzelteilNr':'4006','Bezeichnung': 'Tretkurbel','Preis':'3.99'},
{ 'EinzelteilNr':'5001','Bezeichnung': 'Steuersatz','Preis':'80.00'},
{ 'EinzelteilNr':'5002','Bezeichnung': 'Steuerrohr','Preis':'12.99'},
{ 'EinzelteilNr':'5003','Bezeichnung': 'Gummigriffe','Preis':'0.99'},
{ 'EinzelteilNr':'5004','Bezeichnung': 'Klingel','Preis':'7.99'},
{ 'EinzelteilNr':'6001','Bezeichnung': 'Sitzrohr','Preis':'4.50'},
{ 'EinzelteilNr':'6002','Bezeichnung': 'Sitzstrebe','Preis':'17.50'},
{ 'EinzelteilNr':'7001','Bezeichnung': 'Ventil','Preis':'6.99'},
{ 'EinzelteilNr':'7002','Bezeichnung': 'Schlauch','Preis':'2.99'},
{ 'EinzelteilNr':'7003','Bezeichnung': 'Schutzblech','Preis':'12.99'},
{ 'EinzelteilNr':'8001','Bezeichnung': 'Sattelstütze','Preis':'15.00'},
{ 'EinzelteilNr':'9001','Bezeichnung': 'Schraube','Preis':'0.25'},
{ 'EinzelteilNr':'9002','Bezeichnung': 'Mutter','Preis':'0.50'},
{ 'EinzelteilNr':'9003','Bezeichnung': 'Bremskabel 100 cm','Preis':'1.50'},
{ 'EinzelteilNr':'9004','Bezeichnung': 'Stromkabel 50 cm' ,'Preis':'2.50'},
{ 'EinzelteilNr':'9005','Bezeichnung': 'Lackierung','Preis':'25.50'}]
my_db = db_communication()
for i in range(len(einzelteile)):
    my_db.insert_data("Einzelteile", einzelteile[i])
my_db.close()

Successfully connected.
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('1001', 'Rückstrahler', '5.00');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('2001', 'Bremsgriff', '9.99');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4001', 'Zahnkranz', '50.00');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4002', 'Kette', '9.00');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4003', 'Kettenblätter', '3.99');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4004', 'Pedale', '8.99');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4005', 'Innenlager', '3.99');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('4006', 'Tretkurbel', '3.99');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`, `Preis`) VALUES ('5001', 'Steuersatz', '80.00');
INSERT INTO `Einzelteile`

In [83]:
konfigurator =[{'KonfigNr':'101','AuftragNr':'1','ArtikelNr':'100002'},
               {'KonfigNr':'102','AuftragNr':'1','ArtikelNr':'200002'},
               {'KonfigNr':'103','AuftragNr':'1','ArtikelNr':'300001'},
               {'KonfigNr':'104','AuftragNr':'1','ArtikelNr':'400003'},
               {'KonfigNr':'105','AuftragNr':'1','ArtikelNr':'500002'},
               {'KonfigNr':'106','AuftragNr':'1','ArtikelNr':'600002'},
               {'KonfigNr':'107','AuftragNr':'1','ArtikelNr':'700002'},
               {'KonfigNr':'108','AuftragNr':'1','ArtikelNr':'800004'}]
                
my_db = db_communication()
for i in range(len(konfigurator)):
    my_db.insert_data("Konfigurator", konfigurator[i])
my_db.close()

Successfully connected.
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('101', '1', '100002');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('102', '1', '200002');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('103', '1', '300001');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('104', '1', '400003');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('105', '1', '500002');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('106', '1', '600002');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('107', '1', '700002');
INSERT INTO `Konfigurator` (`KonfigNr`, `AuftragNr`, `ArtikelNr`) VALUES ('108', '1', '800004');
Connection closed


In [85]:
stueckliste = [{'ArtNr': '100001','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '100001','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '100001','EinzelteilNr':'9004','Anzahl':'45'},
               {'ArtNr': '100002','EinzelteilNr':'9001','Anzahl':'4'},
               {'ArtNr': '100002','EinzelteilNr':'9002','Anzahl':'4'},
               {'ArtNr': '100002','EinzelteilNr':'9004','Anzahl':'55'},
               {'ArtNr': '200001','EinzelteilNr':'2001','Anzahl':'2'},
               {'ArtNr': '200001','EinzelteilNr':'9001','Anzahl':'4'},
               {'ArtNr': '200001','EinzelteilNr':'9002','Anzahl':'4'},
               {'ArtNr': '200001','EinzelteilNr':'9003','Anzahl':'35'},
               {'ArtNr': '200002','EinzelteilNr':'2001','Anzahl':'2'},
               {'ArtNr': '200002','EinzelteilNr':'9001','Anzahl':'5'},
               {'ArtNr': '200002','EinzelteilNr':'9002','Anzahl':'5'},
               {'ArtNr': '200002','EinzelteilNr':'9003','Anzahl':'40'},
               {'ArtNr': '200003','EinzelteilNr':'2001','Anzahl':'2'},
               {'ArtNr': '200003','EinzelteilNr':'9001','Anzahl':'6'},
               {'ArtNr': '200003','EinzelteilNr':'9002','Anzahl':'6'},
               {'ArtNr': '200003','EinzelteilNr':'9003','Anzahl':'42'},
               {'ArtNr': '200004','EinzelteilNr':'2001','Anzahl':'2'},
               {'ArtNr': '200004','EinzelteilNr':'9001','Anzahl':'5'},
               {'ArtNr': '200004','EinzelteilNr':'9002','Anzahl':'5'},
               {'ArtNr': '200004','EinzelteilNr':'9003','Anzahl':'37'},
               {'ArtNr': '300001','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '300001','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '300002','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '300002','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '300003','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '300003','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '300004','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '300004','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '300005','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '300005','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '400001','EinzelteilNr':'4001','Anzahl':'1'},
               {'ArtNr': '400001','EinzelteilNr':'4002','Anzahl':'1'},
               {'ArtNr': '400001','EinzelteilNr':'4003','Anzahl':'1'},
               {'ArtNr': '400001','EinzelteilNr':'4004','Anzahl':'2'},
               {'ArtNr': '400001','EinzelteilNr':'4005','Anzahl':'1'},
               {'ArtNr': '400001','EinzelteilNr':'4006','Anzahl':'2'},
               {'ArtNr': '400001','EinzelteilNr':'9001','Anzahl':'17'},
               {'ArtNr': '400001','EinzelteilNr':'9002','Anzahl':'17'},               
               {'ArtNr': '400002','EinzelteilNr':'4001','Anzahl':'1'},
               {'ArtNr': '400002','EinzelteilNr':'4002','Anzahl':'1'},
               {'ArtNr': '400002','EinzelteilNr':'4003','Anzahl':'1'},
               {'ArtNr': '400002','EinzelteilNr':'4004','Anzahl':'2'},
               {'ArtNr': '400002','EinzelteilNr':'4005','Anzahl':'1'},
               {'ArtNr': '400002','EinzelteilNr':'4006','Anzahl':'2'},
               {'ArtNr': '400002','EinzelteilNr':'9001','Anzahl':'17'},
               {'ArtNr': '400002','EinzelteilNr':'9002','Anzahl':'17'},               
               {'ArtNr': '400003','EinzelteilNr':'4001','Anzahl':'1'},
               {'ArtNr': '400003','EinzelteilNr':'4002','Anzahl':'1'},
               {'ArtNr': '400003','EinzelteilNr':'4003','Anzahl':'1'},
               {'ArtNr': '400003','EinzelteilNr':'4004','Anzahl':'2'},
               {'ArtNr': '400003','EinzelteilNr':'4005','Anzahl':'1'},
               {'ArtNr': '400003','EinzelteilNr':'4006','Anzahl':'2'},
               {'ArtNr': '400003','EinzelteilNr':'9001','Anzahl':'17'},
               {'ArtNr': '400003','EinzelteilNr':'9002','Anzahl':'17'},               
               {'ArtNr': '500001','EinzelteilNr':'5001','Anzahl':'1'},
               {'ArtNr': '500001','EinzelteilNr':'5002','Anzahl':'1'},
               {'ArtNr': '500001','EinzelteilNr':'5003','Anzahl':'2'},
               {'ArtNr': '500001','EinzelteilNr':'5004','Anzahl':'1'},
               {'ArtNr': '500001','EinzelteilNr':'9001','Anzahl':'5'},
               {'ArtNr': '500001','EinzelteilNr':'9002','Anzahl':'5'},               
               {'ArtNr': '500002','EinzelteilNr':'5001','Anzahl':'1'},
               {'ArtNr': '500002','EinzelteilNr':'5002','Anzahl':'1'},
               {'ArtNr': '500002','EinzelteilNr':'5003','Anzahl':'2'},
               {'ArtNr': '500002','EinzelteilNr':'5004','Anzahl':'1'},
               {'ArtNr': '500002','EinzelteilNr':'9001','Anzahl':'5'},
               {'ArtNr': '500002','EinzelteilNr':'9002','Anzahl':'5'},              
               {'ArtNr': '500003','EinzelteilNr':'5001','Anzahl':'1'},
               {'ArtNr': '500003','EinzelteilNr':'5002','Anzahl':'1'},
               {'ArtNr': '500003','EinzelteilNr':'5003','Anzahl':'2'},
               {'ArtNr': '500003','EinzelteilNr':'5004','Anzahl':'1'},
               {'ArtNr': '500003','EinzelteilNr':'9001','Anzahl':'5'},
               {'ArtNr': '500003','EinzelteilNr':'9002','Anzahl':'5'},            
               {'ArtNr': '600001','EinzelteilNr':'6001','Anzahl':'1'},
               {'ArtNr': '600001','EinzelteilNr':'6002','Anzahl':'1'},
               {'ArtNr': '600001','EinzelteilNr':'9001','Anzahl':'8'},
               {'ArtNr': '600001','EinzelteilNr':'9002','Anzahl':'8'},
               {'ArtNr': '600001','EinzelteilNr':'9005','Anzahl':'1'},
               {'ArtNr': '600002','EinzelteilNr':'6001','Anzahl':'1'},
               {'ArtNr': '600002','EinzelteilNr':'6002','Anzahl':'1'},
               {'ArtNr': '600002','EinzelteilNr':'9001','Anzahl':'8'},
               {'ArtNr': '600002','EinzelteilNr':'9002','Anzahl':'8'},
               {'ArtNr': '600002','EinzelteilNr':'9005','Anzahl':'1'},
               {'ArtNr': '600003','EinzelteilNr':'6001','Anzahl':'1'},
               {'ArtNr': '600003','EinzelteilNr':'6002','Anzahl':'1'},
               {'ArtNr': '600003','EinzelteilNr':'9001','Anzahl':'8'},
               {'ArtNr': '600003','EinzelteilNr':'9002','Anzahl':'8'},
               {'ArtNr': '600003','EinzelteilNr':'9005','Anzahl':'1'},
               {'ArtNr': '700001','EinzelteilNr':'7001','Anzahl':'2'},
               {'ArtNr': '700001','EinzelteilNr':'7002','Anzahl':'2'},
               {'ArtNr': '700001','EinzelteilNr':'7003','Anzahl':'2'},
               {'ArtNr': '700001','EinzelteilNr':'9001','Anzahl':'6'},
               {'ArtNr': '700001','EinzelteilNr':'9002','Anzahl':'6'},
               {'ArtNr': '700002','EinzelteilNr':'7001','Anzahl':'2'},
               {'ArtNr': '700002','EinzelteilNr':'7002','Anzahl':'2'},
               {'ArtNr': '700002','EinzelteilNr':'7003','Anzahl':'2'},
               {'ArtNr': '700002','EinzelteilNr':'9001','Anzahl':'6'},
               {'ArtNr': '700002','EinzelteilNr':'9002','Anzahl':'6'},
               {'ArtNr': '700003','EinzelteilNr':'7001','Anzahl':'2'},
               {'ArtNr': '700003','EinzelteilNr':'7002','Anzahl':'2'},
               {'ArtNr': '700003','EinzelteilNr':'7003','Anzahl':'2'},
               {'ArtNr': '700003','EinzelteilNr':'9001','Anzahl':'6'},
               {'ArtNr': '700003','EinzelteilNr':'9002','Anzahl':'6'},               
               {'ArtNr': '800001','EinzelteilNr':'8001','Anzahl':'1'},
               {'ArtNr': '800001','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '800001','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '800002','EinzelteilNr':'8001','Anzahl':'1'},
               {'ArtNr': '800002','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '800002','EinzelteilNr':'9002','Anzahl':'3'},
               {'ArtNr': '800003','EinzelteilNr':'8001','Anzahl':'1'},
               {'ArtNr': '800003','EinzelteilNr':'9001','Anzahl':'3'},
               {'ArtNr': '800003','EinzelteilNr':'9002','Anzahl':'3'}]
               

my_db = db_communication()
for i in range(len(stueckliste)):
     my_db.insert_data("Stueckliste", stueckliste[i])
my_db.close()

Successfully connected.
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9001', '3');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9002', '3');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100001', '9004', '45');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9001', '4');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9002', '4');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('100002', '9004', '55');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('200001', '2001', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('200001', '9001', '4');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('200001', '9002', '4');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('200001', '9003', '35');
INSERT INTO `Stueckliste` (`ArtNr

INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700001', '7002', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700001', '7003', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700001', '9001', '6');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700001', '9002', '6');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700002', '7001', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700002', '7002', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700002', '7003', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700002', '9001', '6');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700002', '9002', '6');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`) VALUES ('700003', '7001', '2');
INSERT INTO `Stueckliste` (`ArtNr`, `EinzelteilNr`, `Anzahl`